# Projet de classification de textes
Nous en sommes maintenant au point où nous devrions pouvoir.. :
* Lire une collection de documents - un *corpus*
* Transformer un texte en données numériques vectorielles à l'aide d'un pipeline
* Créer un classificateur
* Adapter/entraîner le classificateur
* Tester le classificateur sur de nouvelles données
* Évaluer les performances

Pour ce projet, nous utiliserons le Cornell University Movie Review polarity dataset v2.0 obtenu à partir de http://www.cs.cornell.edu/people/pabo/movie-review-data/.

Dans cet exercice, nous essaierons de développer un modèle de classification - c'est-à-dire que nous essaierons de prédire les étiquettes Positif/Négatif en nous basant uniquement sur le contenu du texte.

## Effectuer les importations et charger le jeu de données
L'ensemble de données contient le texte de 2000 critiques de films. 1000 sont positives, 1000 sont négatives, et le texte a été prétraité sous forme de fichier délimité par des tabulations.

In [1]:
import numpy as np
import pandas as pd

# VOTRE CODE ICI

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


In [2]:
len(df)

2000

### Examinez un avis typique. Celui-ci est qualifié de "négatif" :

In [3]:
from IPython.display import Markdown, display
display(Markdown('> '+df['review'][0]))

> how do films like mouse hunt get into theatres ? 
isn't there a law or something ? 
this diabolical load of claptrap from steven speilberg's dreamworks studio is hollywood family fare at its deadly worst . 
mouse hunt takes the bare threads of a plot and tries to prop it up with overacting and flat-out stupid slapstick that makes comedies like jingle all the way look decent by comparison . 
writer adam rifkin and director gore verbinski are the names chiefly responsible for this swill . 
the plot , for what its worth , concerns two brothers ( nathan lane and an appalling lee evens ) who inherit a poorly run string factory and a seemingly worthless house from their eccentric father . 
deciding to check out the long-abandoned house , they soon learn that it's worth a fortune and set about selling it in auction to the highest bidder . 
but battling them at every turn is a very smart mouse , happy with his run-down little abode and wanting it to stay that way . 
the story alternates between unfunny scenes of the brothers bickering over what to do with their inheritance and endless action sequences as the two take on their increasingly determined furry foe . 
whatever promise the film starts with soon deteriorates into boring dialogue , terrible overacting , and increasingly uninspired slapstick that becomes all sound and fury , signifying nothing . 
the script becomes so unspeakably bad that the best line poor lee evens can utter after another run in with the rodent is : " i hate that mouse " . 
oh cringe ! 
this is home alone all over again , and ten times worse . 
one touching scene early on is worth mentioning . 
we follow the mouse through a maze of walls and pipes until he arrives at his makeshift abode somewhere in a wall . 
he jumps into a tiny bed , pulls up a makeshift sheet and snuggles up to sleep , seemingly happy and just wanting to be left alone . 
it's a magical little moment in an otherwise soulless film . 
a message to speilberg : if you want dreamworks to be associated with some kind of artistic credibility , then either give all concerned in mouse hunt a swift kick up the arse or hire yourself some decent writers and directors . 
this kind of rubbish will just not do at all . 


## Vérifier les valeurs manquantes :
Nous avons intentionnellement inclus des enregistrements avec des données manquantes. Certains ont des valeurs NaN, d'autres ont des chaînes courtes composées uniquement d'espaces. Cela peut se produire si un évaluateur a refusé de fournir un commentaire avec son évaluation. Nous allons montrer deux façons d'utiliser pandas pour identifier et supprimer les enregistrements contenant des données vides.
* Les enregistrements NaN sont traités efficacement avec [.isnull()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.isnull.html) et [.dropna()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html).
* Les chaînes de caractères qui ne contiennent que des espaces peuvent être traitées avec [.isspace()](https://docs.python.org/3/library/stdtypes.html#str.isspace), [.itertuples()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.itertuples.html), et [.drop()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html).

### Détecter et supprimer les valeurs NaN :

In [4]:
# Check for the existence of NaN values in a cell:
# VOTRE CODE ICI

label      0
review    35
dtype: int64

In [5]:
# SUPPRESSION DE NaN

1965

### Détecter et supprimer les chaînes vides
Techniquement, nous avons à faire à des chaînes de caractères "whitespace only". Si le fichier .tsv original avait contenu des chaînes vides, pandas **.read_csv()** aurait attribué des valeurs NaN à ces cellules par défaut.

Afin de détecter ces chaînes, nous devons itérer sur chaque ligne du DataFrame. La méthode pandas **.itertuples()** est un bon outil pour cela car elle permet d'accéder à chaque champ. Par souci de concision, nous attribuerons les noms `i`, `lb` et `rv` aux colonnes `index`, `label` et `review`.

In [6]:
blanks = []  # start with an empty list

for i,lb,rv in df.itertuples():  # iterate over the DataFrame
    if type(rv)==str:            # avoid NaN values
        if rv.isspace():         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list
        
print(len(blanks), 'blanks: ', blanks)

27 blanks:  [57, 71, 147, 151, 283, 307, 313, 323, 343, 351, 427, 501, 633, 675, 815, 851, 977, 1079, 1299, 1455, 1493, 1525, 1531, 1763, 1851, 1905, 1993]


Ensuite, nous allons passer notre liste de numéros d'index à la méthode **.drop()**, et mettre `inplace=True` pour rendre le changement permanent.

In [7]:
# VOTRE CODE ICI

1938

C'est formidable ! Nous avons supprimé 62 enregistrements des 2000 originaux. Poursuivons l'analyse.

## Jetez un coup d'œil à la colonne `label` :

In [8]:
# VOTRE CODE ICI

neg    969
pos    969
Name: label, dtype: int64

## Diviser les données en ensembles d'entrainement et de test :

In [9]:
from sklearn.model_selection import train_test_split

# VOTRE CODE ICI

## Construire des pipelines pour vectoriser les données, puis entraîner et ajuster un modèle
Maintenant que nous disposons de jeux de données à entraîner et à tester, nous allons développer une sélection de pipelines, chacun avec un modèle différent. Pour la vectorisation vous utiliserez TfidfVectorizer

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

# VOTRE CODE ICI

## Introduire les données d'apprentissage dans le premier pipeline

In [11]:
# VOTRE ENTRAINEMENT

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

## Exécuter les prédictions et analyser les résultats

In [ ]:
# Form a prediction set
# VOTRE CODE ICI

In [16]:
# Report the confusion matrix
from sklearn import metrics
# VOTRE CODE ICI

[[287  21]
 [130 202]]


In [17]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

         neg       0.69      0.93      0.79       308
         pos       0.91      0.61      0.73       332

    accuracy                           0.76       640
   macro avg       0.80      0.77      0.76       640
weighted avg       0.80      0.76      0.76       640



In [18]:
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.7640625


### Ensuite, alimenter la méthode `predict()` du modèle avec de nouvelles données

In [92]:
myreview = "A movie I really wanted to love was terrible. \
I'm sure the producers had the best intentions, but the execution was lacking."

In [101]:
# Use this space to write your own review. Experiment with different lengths and writing styles.
myreview = "I love the movie !"




In [102]:
print(text_clf_nb.predict([myreview]))  # be sure to put "myreview" inside square brackets

['pos']


# Soyez force de proposition et trouver d'autres façons d'optimiser le modèle si vous voulez aller plus loin.
# NB : pas obligatoire, juste pour votre curiosité.